In [1]:
import pandas as pd
import numpy as np

data = pd.read_csv('Results/RequiredDataSet.csv',header=None, names=['booking_date','channel','property_id','room_name','room_id','check_in','check_out','room_rate','booking_status'])
# data.head()


In [2]:
Results = pd.read_excel('Results/DataSetWithStayDate.xlsx')

In [ ]:
# Results.head()

In [ ]:
del Results['Old_Room_rate']
# Results.head()

In [ ]:
scrapyData = pd.read_csv('Results/eventData2015.csv',header=None, names=['Day','Stay_date','Event','HolidayIn'])

In [ ]:
scrapeResult = scrapyData.dropna()


In [ ]:
# scrapeResult.head()

In [ ]:
scrapeResult['Stay_date'] =   '2015-' + scrapeResult['Stay_date'] 
# scrapeResult.head()

In [ ]:
scrapeResult['Stay_date'] = pd.to_datetime(scrapeResult.Stay_date)
#pd.to_datetime('02/Jan/2015:08:07:32', format='%d/%b/%Y:%H:%M:%S')

In [ ]:
# scrapeResult.head()

In [ ]:
del scrapeResult['HolidayIn']
# scrapeResult.head()

In [ ]:
newResults = Results.filter(['Property_id','Stay_date','Booking_status','Room_name','Room_details','Room_rate','Channel'], axis=1)
# newResults.head()

In [ ]:
newResults = newResults[(newResults.Booking_status == 'COMPLETED') | (newResults.Booking_status == 'CONFIRMED')]
# newResults.head()

In [ ]:
newResults = newResults[(newResults.Stay_date >= '2015-01-01') & (newResults.Stay_date <= '2015-12-31')]
# newResults.head()

In [ ]:
newResults['Stay_date'] = pd.to_datetime(newResults.Stay_date)
# newResults.head()

In [ ]:
del newResults['Room_details']

In [ ]:
del scrapeResult['Day']
# scrapeResult.head()

In [ ]:
res = pd.merge(left=newResults, left_on='Stay_date',right=scrapeResult, right_on='Stay_date', how='left')

#res = pd.merge(newResults, scrapeResult, on='Stay_date', how='left')

In [ ]:
res = res.sort_values(by = 'Stay_date', ascending=True)
# res.head()

In [ ]:
res1 = res.drop_duplicates()
# res1.head()

In [ ]:
res1 = res1.reset_index()
# res1.head()

In [ ]:
del res1['index']
# res1.head()

In [ ]:
res1 = res1.fillna('-')

In [ ]:
events = res1[(res1.Event != '-') ]
# events

In [ ]:
events['totalRevenue'] = events.groupby(['Property_id','Stay_date'])['Room_rate'].transform('mean')
# events

In [ ]:
events_tr = events["totalRevenue"].mean()
events_tr

In [ ]:
events['room_count'] = events.groupby(['Property_id','Stay_date'])['Room_rate'].transform('count')
# events.head()

In [ ]:
events['ADR'] = events.apply(lambda events: (events['totalRevenue'] / events['room_count'] ) , axis=1)


In [ ]:
events_adr = events["ADR"].unique().mean()
events_adr

In [ ]:
# events

In [ ]:
nonevents = res1[(res1.Event == '-') ]
# nonevents

In [ ]:
nonevents['totalRevenue'] = nonevents.groupby(['Property_id','Stay_date'])['Room_rate'].transform('mean')
# nonevents.head()

In [ ]:
nonevents_tr = nonevents["totalRevenue"].mean()
nonevents_tr

In [ ]:
nonevents['room_count'] = nonevents.groupby(['Property_id','Stay_date'])['Room_rate'].transform('count')
# nonevents.head()

In [ ]:
nonevents['ADR'] = nonevents.apply(lambda nonevents: (nonevents['totalRevenue'] / nonevents['room_count'] ) , axis=1)
# nonevents.head()

In [ ]:
nonevents_adr = nonevents["ADR"].unique().mean()
nonevents_adr

In [ ]:
# nonevents